In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

In [ ]:
#Read the PAMPA data

pampa_train = pd.read_csv('raw_data/pampa/pampa_papp_from_article.csv')

pampa_val_bi = pd.read_csv('raw_data/pampa/papp_pampa_high_low.csv', delimiter = ',')
pampa_val_cont= pd.read_csv('raw_data/pampa/papp_pampa_speed.csv', delimiter=',')

In [ ]:
#Select columns we need
pampa_train = pampa_train[['PUBCHEM_EXT_DATASOURCE_SMILES','Phenotype','Permeability']]
pampa_val_bi = pampa_val_bi[['SMILES','Papp(PAMPA)(qualitative)']]
pampa_val_cont = pampa_val_cont[['SMILES','Papp(PAMPA) {measured, converted}']]

In [ ]:
#Categorize cont_bbb data as high or low permeable

def compare_values_val(row):
    if row['Papp(PAMPA) {measured, converted}'] > 4:
        return 'high'
    elif row['Papp(PAMPA) {measured, converted}'] < 2:
        return 'low'
    else:
        return '?'

pampa_val_cont['status_pampa'] = pampa_val_cont.apply(compare_values_val, axis=1)


In [ ]:
#Categorize training data as high or low permeable

def compare_values_train(row):

    if row['Permeability'] > 4:
        return 'high'
    elif row['Permeability'] < 2:
        return 'low'
    else:
        return '?'

pampa_train['Permeability'] = pampa_train['Permeability'].astype(float)
pampa_train['status_pampa'] = pampa_train.apply(compare_values_train, axis=1)


In [ ]:
#Rename columns
pampa_train.rename(columns={'PUBCHEM_EXT_DATASOURCE_SMILES':'SMILES_raw'}, inplace=True)
pampa_val_cont.rename(columns={'SMILES':'SMILES_raw'}, inplace=True)
pampa_val_bi.rename(columns={'SMILES':'SMILES_raw','Papp(PAMPA)(qualitative)':"status_pampa"}, inplace=True)

In [ ]:
#Keep only compounds with class
print(f'Full dataset: {len(pampa_train)}')
pampa_train_filt = pampa_train[pampa_train['status_pampa'].isin(['high', 'low'])]
print(f'Compounds with class: {len(pampa_train_filt)}')

In [ ]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [ ]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [ ]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [ ]:
#Create connectivity inchi

def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [ ]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_pampa'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(f'After contradicting removed: {len(df)}')
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    print(f'After duplicated removed: {len(df)}')
    #df['inchi_stereo'].values_counts

    return df

In [ ]:
def class_code(df):
    df['status_pampa'] = df['status_pampa'].replace({'high': 1, 'low': 0})

    return df

In [ ]:
#Prepare training data

df = pampa_train_filt

df = remove_nan_smiles(df)

df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_pampa_train.csv')


In [ ]:
#Prepare binary validation data

pampa_val_bi = remove_nan_smiles(pampa_val_bi)

pampa_val_bi_sd = standardize_workflow(pampa_val_bi)
pampa_val_bi_valid = missing_inchi(pampa_val_bi_sd)
pampa_val_bi_connectivity_inchi = create_connectivity_inchi(pampa_val_bi_valid)
pampa_val_bi_no_duplicates = remove_duplicates(pampa_val_bi_connectivity_inchi)
pampa_val_bi_class_coded = class_code(pampa_val_bi_no_duplicates)


In [ ]:
#Prepare continous validation data

pampa_val_cont = remove_nan_smiles(pampa_val_cont)

pampa_val_cont_sd = standardize_workflow(pampa_val_cont)
pampa_val_cont_valid = missing_inchi(pampa_val_cont_sd)
pampa_val_cont_connectivity_inchi = create_connectivity_inchi(pampa_val_cont_valid)
pampa_val_cont_no_duplicates = remove_duplicates(pampa_val_cont_connectivity_inchi)
filtered_cont_pampa = pampa_val_cont_no_duplicates[~(pampa_val_cont_no_duplicates['status_pampa'] == '?')]

pampa_val_cont_class_coded = class_code(filtered_cont_pampa)



Combining validation sets

In [ ]:
df_combined_val = pd.concat([pampa_val_bi_class_coded,pampa_val_cont_class_coded], ignore_index=True, join="inner")

In [ ]:
df_val_no_dup = remove_duplicates(df_combined_val)
df_val_no_dup.to_csv('val_data/kadar_pampa_val.csv', index=True)